# Majority voting
This jupyter notebook demonstrates how the [agent-based model](https://doi.org/10.5281/zenodo.7767125) of majority voting works by walking you through the stages of the model. The agent-based model is basically an adaptation from the Condorcetian framework (see [Wikipedia](https://en.wikipedia.org/wiki/Condorcet%27s_jury_theorem) or [The Stanford Encyclopedia of Philosophy](https://plato.stanford.edu/archives/spr2023/entries/jury-theorems/)) which operationalizes the following assumptions:
1. **The Assumption of Plural Interests.** For each agent, there exists some agent-relative measure of success or correctness – typically referred to as the agent’s interest or values. 
2.	**The Group-relative Competence Assumption.** For each of the two groups (masses and elites), each member’s belief about the right alternative is true with probability greater than chance level.
3. **The Social Influence Assumption.** (a) The initial beliefs of all voters are probabilistically independent (given the true state of the world). (b) The final beliefs of voters can be influenced by other voters. 
4. **The Sincerity Assumption.** All voters vote for the alternative they believe to be the right alternative.

In particular, an instantiation of the agent-based model, called `Community`, consists of the following building blocks:
- A set of agents $N$,
- A partition of $N$ into elites $E$ and masses $M$,
- Group-relative competences $p_e$ and $p_m$ (in $[0,1]$) for elites and masses, respectively,
- For each agent $i \in N$:
    - Her membership in either the elites ($E$) or the masses ($M$),
    - Her individual competence $p_i\in [0,1]$ (which equals either $p_e$ or $p_m$, depending on the group she belongs to)
    - Her social environment $S_i \subseteq N$.

We start with loading some packages and scripts:

In [1]:
%run jupyter.py

## 1. A simple example
### 1.0 Initial stage
Let us start with considering a simple example of a community with only 10 agents. More explicitly, we set the parameters as follows:
- A set of agents $N=\{0, \ldots, 9\}$,
- A partition of $N$ into elites $E=\{0, 1,2\}$ and masses $M=\{3, \ldots, 9\}$,
- Group-relative competences $p_e=0.8$ and $p_m=0.6$ for elites and masses, respectively,
- For each agent $i \in N$:
    - Her social environment $S_i \subseteq N$ (where $|S_i|=6$) follows from the edges in the network that is generated using preferential attachment and homophilic selection.

The simple community is depicted in the picture below. Let me explain the picture:
- Nodes:
    - The size is proportional to the in-degree, so that bigger nodes are more influential.
- Edges:
    - A directed edge from a source node to a target node indicates that the source node influences the target node. Every node is influenced by 6 others. 
- Colors:
    - The color of the *nodes* represents whether they are <font color="#FFC107">elites</font> or <font color="#9C27B0">masses</font>.
    - The colors of the *edges* represent whether the source nodes are <font color="#FFC107">elites</font> or <font color="#9C27B0">masses</font>.  

All the pictures are **interactive**: you can move nodes around!

In [2]:
community_simple = Community(
    number_of_nodes=10, 
    number_of_elites=3, 
    elite_competence=0.8, 
    mass_competence=0.6,
)
visualize(community_simple).show("community_simple_initial.html")

community_simple_initial.html


### 1.1 Opinions
In the first stage, the agents form their own individual opinion. Agent $i$'s individual competence $p_i$ expresses the probability that she forms the right opinion, i.e., that her beliefs about the alternative that is in her own best interest is *correct*.

- Colors in the picture:
    - The color of the *nodes* represents their **opinion** favours the <font color="#FFC107">elites</font> or <font color="#9C27B0">masses</font>.
    - The colors of the *edges* represent the color of the source nodes.  

Let's see how this goes in the simple community. Did anyone form the wrong opinion? Can you figure out why?

In [3]:
community_simple.update_votes()
visualize(com=community_simple, color_type="opinion").show("community_simple_opinions.html")

community_simple_opinion.html


### 1.2 Social influence
In the second stage, the agents form their voting decision. Agent $i$'s voting decision is determined by her own opinion and the opinions of the agents in her social environment $S_i$. 

- Colors in the picture:
    - The color of the *nodes* represents their **vote** favours the <font color="#FFC107">elites</font> or <font color="#9C27B0">masses</font>.
    - The colors of the *edges* represent the color of the source nodes.  

Let's see how this goes in the simple community. Did anyone form the wrong voting decision? Can you figure out why?

In [4]:
visualize(com=community_simple, color_type="vote").show("community_simple_votes.html")

community_simple_vote.html


### 1.3 The vote
The outcome is probably no surprise to you at this point, but let's see the outcome of majority voting:

In [5]:
votes_for_elites = sum(
    [1 for node in community_simple.nodes if community_simple.network.nodes[node]["vote"] == vote_for_elites]
)
votes_for_masses = community_simple.number_of_nodes - votes_for_elites
print(f"Elites: {votes_for_elites} | Masses: {votes_for_masses}")

Elites: 10 | Masses: 0


## 2. A more complex example
### 2.0 Initial stage
Let us now consider a bigger community with 100 agents, where 30 are elites. More explicitly, we set the parameters as follows:
- A set of agents $N=\{0, \ldots, 90\}$,
- A partition of $N$ into elites $E=\{0, \ldots, 29\}$ and masses $M=\{30, \ldots, 99\}$,
- Group-relative competences $p_e=0.8$ and $p_m=0.6$ for elites and masses, respectively,
- For each agent $i \in N$:
    - Her social environment $S_i \subseteq N$ (where $|S_i|=6$) follows from the edges in the network that is generated using preferential attachment and homophilic selection.

The community is depicted in the picture below. Let me explain the colors in the picture:
- Colors:
    - The color of the *nodes* represents whether they are <font color="#FFC107">elites</font> or <font color="#9C27B0">masses</font>.
    - The colors of the *edges* represent the color of the source nodes.

In [6]:
community = Community(
    number_of_nodes=100, 
    number_of_elites=30, 
    elite_competence=0.8, 
    mass_competence=0.6,
)
visualize(community).show("community_initial.html")

community_initial.html


The network is generated by homophilic selection and by preferential attachment. You can roughly see that the network is generated by preferential attachment because most agents have little influence, while a few have a lot of influence. 

Let us compute the number of agents that are influenced by the most influential agent:

In [7]:
max_in_degree = sorted(community.network.in_degree, key=lambda x: x[1], reverse=True)[0]
print(f"Node {max_in_degree[0]} has the most influence: they influence {max_in_degree[1]} others!")

Node 71 has the most influence: they influence 36 others!


### 2.1 Opinions
In the first stage, the agents form their own individual opinion. Agent $i$'s individual competence $p_i$ expresses the probability that she forms the right opinion, i.e., that her beliefs about the alternative that is in her own best interest is *correct*.

- Colors:
    - The color of the *nodes* represents their **opinion** favours the <font color="#FFC107">elites</font> or <font color="#9C27B0">masses</font>.
    - The colors of the *edges* represent the color of the source nodes.  

Let's see how this goes in the complex community. Did anyone form the wrong opinion? Can you figure out why?

In [8]:
community.update_votes()
visualize(com=community, color_type="opinion").show("community_opinions.html")

community_opinions.html


Given the size of the community, it is hard to see the distribution of opinions, so let's compute this:

In [9]:
opinions_for_elites = sum(
    [1 for node in community.nodes if community.network.nodes[node]["opinion"] == vote_for_elites]
)
opinion_for_masses = community.number_of_nodes - opinions_for_elites
print(f"Opinions: Elites: {opinions_for_elites} | Masses: {opinion_for_masses}")

Opinions: Elites: 47 | Masses: 53


### 2.2 Social influence
In the second stage, the agents form their voting decision. Agent $i$'s voting decision is determined by her own opinion and the opinions of the agents in her social environment $S_i$. 

Let's see how this goes in the complex community. Did anyone form the wrong voting decision? Can you figure out why?

In [13]:
visualize(com=community, color_type="vote").show("community_votes.html")

community_votes.html


### 2.3 The vote
The outcome is probably no surprise to you at this point, but let's see the distribution of the votes:

In [11]:
votes_for_elites = sum(
    [1 for node in community.nodes if community.network.nodes[node]["vote"] == vote_for_elites]
)
votes_for_masses = community.number_of_nodes - votes_for_elites
print(f"Votes: Elites: {votes_for_elites} | Masses: {votes_for_masses}")

Votes: Elites: 46 | Masses: 54


Let's take a step back and see how the distribution changed from the initial distribution of elites and masses via the distribution of opinions to the distribution of votes:

In [12]:
print(f"Initial distribution: Elites: {community.number_of_elites} | Masses: {community.number_of_mass}\n"
     f"Opinion distribution: Elites: {opinions_for_elites} | Masses: {opinion_for_masses}\n"
     f"Votes distribition: Elites: {votes_for_elites} | Masses: {votes_for_masses}")


Initial distribution: Elites: 30 | Masses: 70
Opinion distribution: Elites: 47 | Masses: 53
Votes distribition: Elites: 46 | Masses: 54


## 3. Thanks
I hope this brief illustration helped to understand the inner workings of the agent-based model of majority voting. Thanks for showing interest!